In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import random
import matplotlib.image as mpimg
from imutils import paths
import sys
import operator

In [3]:
!pip install rich
!pip install hydra-core
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.5 MB/s 
     |████████████████████████████████| 51 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 7.2 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 117 kB 56.6 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=4f9f07a54dfff645aec7e32c60da504ebd876652e571b9bfb9341538eb565eb5
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 8.1 MB/s 
     |████████████████████████████████| 5.9 MB 45.7 MB/s 
     |████████████████████████████████| 419 kB 50.2 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [4]:
sys.path.insert(0, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation")
sys.path.insert(1, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation/src/utils")

In [5]:
import pytorch_lightning as pl
from src.utils.display_image_from_path import img_show
from src.utils.display_image_from_array import img_show_from_arr
from src.utils.delete_border_lines import delete_border_lines
from src.utils.take_pixels_whose_neighbors_are_zero import next_neighbor_as_zero
from src.utils.get_row_heights import get_row_heights
from src.utils.get_col_widths import get_col_widths
from src.utils.erase_short_lines import erase_lines_with_small_width
from src.utils.erase_small_lines import erase_lines_with_small_height
from src.utils.find_image_page_type import find_image_page_type
from src.utils.create_row_patches_from_lines_y_coordinates import create_row_patches
from src.utils.get_unique_cols_from_row_patches import create_unique_patch_cols
from src.utils.create_row_patch_mapping import get_patch_map
from src.utils.create_bboxes import create_bbox_list
from src.utils.create_line_images_from_bbox import create_line_image_from_bbox
from src.utils.create_row_images_from_bbox_list import create_row_images
from src.utils.get_bboxes_from_mask import get_bboxes_from_mask
from src.utils.save_line_images import save_line_images
from src.utils.display_and_save_line_images import display_and_save_line_images

In [16]:
def preprocess_erasing(mask):
  mask = delete_border_lines(mask=mask, x_ratio=0.05, y_ratio=0.1)
  freq_y = []
  h, w, d = mask.shape
  for j in range(w):
    counter = 0
    for i in range(h):
      if mask[i][j][0] != 0:
        counter += 1
    freq_y.append(counter)
  
  freq_ynz = []
  for i in range(len(freq_y)):
    if freq_y[i] != 0:
      freq_ynz.append(freq_y[i])

  mean_freq_y = np.mean(np.array(freq_ynz))
  for i in range(len(freq_y)):
    if freq_y[i] < 0.8 * mean_freq_y or freq_y[i] > 2 * mean_freq_y:
      for j in range(h):
        mask[j][i] = 0

  freq_x = []
  for i in range(h):
    counter = 0
    for j in range(w):
      if mask[i][j][0] != 0:
        counter += 1
    freq_x.append(counter)

  freq_xnz = []
  for i in range(len(freq_x)):
    if freq_x[i] != 0:
      freq_xnz.append(freq_x[i])

  mean_freq_x = np.mean(np.array(freq_xnz))

  for i in range(len(freq_x)):
    if freq_x[i] < 0.3 * mean_freq_x:
      for j in range(w):
        mask[i][j] = 0
    
  return mask

In [ ]:
img_path = "/content/drive/MyDrive/Images/T 2438---0045.jpeg"
img = cv2.imread(img_path)
img_mask = cv2.imread("/content/drive/MyDrive/output/test_page_predicted_masks/T 2438---0045_mask_240x240.png")

img_show_from_arr(img)
img_show_from_arr(img_mask)

In [18]:
mask = img_mask.copy()
mask = preprocess_erasing(mask)

In [ ]:
img_show_from_arr(img_mask)
img_show_from_arr(mask)

In [ ]:
bboxes = get_bboxes_from_mask(mask, 0.05, 0.1, 0.7, 0.8, 0.01, 0.01, 0.1, 0.8)
bboxes

In [28]:
def calculate_safety_gap_xy(bboxes, safety_gap_x_ratio, safety_gap_y_ratio):
  row_heights = []
  col_widths = []
  for box in bboxes:
    x1, y1, x2, y2 = box
    col_widths.append(abs(x1-x2))
    row_heights.append(abs(y1-y2))
  mean_row_height = np.mean(np.array(row_heights))
  mean_col_width = np.mean(np.array(col_widths))
  safety_gap_x = safety_gap_x_ratio * mean_col_width
  safety_gap_y = safety_gap_y_ratio * mean_row_height
  return int(safety_gap_x), int(safety_gap_y)

In [29]:
safety_gap_xy = calculate_safety_gap_xy(bboxes, 0.05, 0.35)
safety_gap_xy

(42, 16)

In [ ]:
display_and_save_line_images(bboxes, img, safety_gap_xy[0], safety_gap_xy[1])